## GGD Case intensity prediction - Part 3 (Censored sets)

### 1. Import modules

In [1]:
# Global
import warnings
warnings.filterwarnings('ignore')

# Preprocessing
from data_preprocessors.balancer import *
from data_preprocessors.data_preparation import *
from data_preprocessors.data_splitting import *
from data_preprocessors.data_transformation import *
from data_preprocessors.feature_addition import *
from data_preprocessors.target_creation import *

import pandas as pd
import pyodbc
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display='diagram')

# Model selection
import shap
import joblib
from sklearn.metrics import median_absolute_error, mean_absolute_percentage_error, mean_squared_error, mean_squared_log_error, r2_score
from tabulate import tabulate

### 2. Import data

In [2]:
# 02-05-2024 censored sets
X_test_censored_featurized = pd.read_pickle('..\\..\\Data\\Splits\\20240502_X_test_censored_featurized.pkl')
X_oos_censored_featurized = pd.read_pickle('..\\..\\Data\\Splits\\20240502_X_oos_censored_featurized.pkl')

In [3]:
# Replace server name if source changes
server = 'AZ-ODB0\ODBWB'
database = 'OnderzoeksDB_WB'

# Create a connection string
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database}'

# Establish connection
connection = pyodbc.connect(connection_string)

# Create SQL query string
query = 'SELECT * FROM Regas_Problematiek'

# Execute the query and fetch the data into a Pandas DataFrame
df_censored_og = pd.read_sql(query, connection)

# Close the connection
connection.close()

In [4]:
# Group by 'Casus' and 'Persoonsnummer' and get the last non-null value from each column
df_censored_og = df_censored_og.groupby(['Casus', 'Persoonsnummer'], as_index=False).last()

# Save column 'casus' as index
df_censored_og.set_index('Casus', inplace=True)

In [5]:
# Get the indices from X_censored_featurized that exist in df_censored_og
common_indices_test = df_censored_og.index.intersection(X_test_censored_featurized.index)
common_indices_oos = df_censored_og.index.intersection(X_oos_censored_featurized.index)

# Store all rows of df_censored_og where 'Casus' is the same as the common indices
df_test_censored = df_censored_og.loc[common_indices_test]
df_oos_censored = df_censored_og.loc[common_indices_oos]

### 3. Select desired rows, columns and corresponding Dtypes

In [6]:
# Select columns from df_og to work with
df_test_censored = column_selector(df_test_censored)
df_oos_censored = column_selector(df_oos_censored)

# Map boolean-like values to True's and False's
df_test_censored = boolean_mapper(df_test_censored)
df_oos_censored = boolean_mapper(df_oos_censored)

# Cast columns to appropriate data types
df_test_censored = dtype_caster(df_test_censored)
df_oos_censored = dtype_caster(df_oos_censored)

### 4. Preprocessing

In [7]:
# Convert dates from original df
df_test_censored['Datum'] = df_test_censored['Datum'].apply(dates_converter)
df_oos_censored['Datum'] = df_oos_censored['Datum'].apply(dates_converter)

# Drop all rows with condition 'Onverzekerde zorg GGD GHOR NL'
df_test_censored.drop(df_test_censored[df_test_censored['Vermoeden_van_problematiek'] == 'Onverzekerde zorg GGD GHOR NL'].index, inplace=True)
df_oos_censored.drop(df_oos_censored[df_oos_censored['Vermoeden_van_problematiek'] == 'Onverzekerde zorg GGD GHOR NL'].index, inplace=True)

# Dropping rows where 'Datum_melding' column is empty (there are none so far)
df_test_censored.dropna(subset=['Datum_melding'], inplace=True)
df_oos_censored.dropna(subset=['Datum_melding'], inplace=True)

# Overwrite the column with the transformed version
df_test_censored['Gemeente'] = df_test_censored['Gemeente'].apply(municipality_mapper).astype('category')
df_oos_censored['Gemeente'] = df_oos_censored['Gemeente'].apply(municipality_mapper).astype('category')

# Drop rows where 'Gemeente' is 'outside_WB'
df_test_censored.drop(df_test_censored[df_test_censored['Gemeente'] == 'outside_WB'].index, inplace=True)
df_oos_censored.drop(df_oos_censored[df_oos_censored['Gemeente'] == 'outside_WB'].index, inplace=True)

### 5. Construct target features

In [8]:
# Add a column indicating a 0 if the 'Datum_afsluiting' column is empty and a 1 if it is not
df_test_censored['Closed'] = df_test_censored['Datum_afsluiting'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_oos_censored['Closed'] = df_oos_censored['Datum_afsluiting'].apply(lambda x: 0 if pd.isnull(x) else 1)

# Add the calculated days to a new column
df_test_censored['Dagen_tot_laatst'] = df_test_censored.apply(lambda x: days_extractor(x.Datum_melding, x.Datum), axis=1)
df_oos_censored['Dagen_tot_laatst'] = df_oos_censored.apply(lambda x: days_extractor(x.Datum_melding, x.Datum), axis=1)

# Add the calculated amount of interventions to a new column
df_test_censored['No_interventions'] = df_test_censored['Datum'].apply(interventions_extractor)
df_oos_censored['No_interventions'] = df_oos_censored['Datum'].apply(interventions_extractor)

In [9]:
# Create y_test_censored and y_oos_censored that store only 'Dagen_tot_laatst' and 'No_interventions' from their corresponding df'2_Selection.ipynb
y_test_censored = df_test_censored[['Closed', 'Dagen_tot_laatst', 'No_interventions']]
y_oos_censored = df_oos_censored[['Closed', 'Dagen_tot_laatst', 'No_interventions']]

# Log transform Days and Interventions, keep 'Closed' as is
y_test_censored_log = y_test_censored['Closed']
y_test_censored_log = pd.concat([y_test_censored_log, y_test_censored[['Dagen_tot_laatst', 'No_interventions']].applymap(lambda x: np.log1p(x))], axis=1)
y_oos_censored_log = y_oos_censored['Closed']
y_oos_censored_log = pd.concat([y_oos_censored_log, y_oos_censored[['Dagen_tot_laatst', 'No_interventions']].applymap(lambda x: np.log1p(x))], axis=1)

### 6. Save splits

In [10]:
# Define list of names equal to variable names
split_names = ['y_test_censored', 'y_oos_censored', 'y_test_censored_log', 'y_oos_censored_log']
splits = [y_test_censored, y_oos_censored, y_test_censored_log, y_oos_censored_log]

# Save all test and oos sets as separate .pkl files
for split in range(len(splits)):
    splits[split].to_pickle(f'..\\..\\Data\\Splits\\20240516_{split_names[split]}.pkl') #RUN THIS ON 16-05-2024
